# 16.DDPG（deep deterministic policy gradient）：深度确定性策略梯度
1. **在线策略算法** 的 **样本效率（sample efficiency）** 都比较低。 **DQN 算法** 虽然为 **离线策略学习**，但是它只能处理 **动作空间有限** 的环境，即使将动作空间离散化，但这会比较粗糙，无法精细控制。
2. ***深度确定性策略梯度（deep deterministic policy gradient，DDPG）*** 结合了 **确定性策略梯度（Deterministic Policy Gradient, DPG）** 与 **DQN 算法** 的思想，通过构造一个 **确定性策略**，来处理 **动作空间无限** 的环境并且是 **离线策略算法**。
3. **DDPG** 也属于一种 **Actor-Critic** 算法。不过之前学习的 REINFORCE、TRPO 和 PPO 学习 **随机性策略**，而 **DDPG** 则学习一个 **确定性策略**。
> **DDPG** 最早由 **Google DeepMind** 于 2015 年在 [Continuous control with deep reinforcement learning](https://arxiv.org/abs/1509.02971#) 中提出，主要用于解决传统 DQN **不能处理连续动作空间** 的问题

## 16.1 DDPG 算法原理

### 确定性策略梯度定理
> 传统**策略梯度方法（PG）** ：
$$J(\theta)=\mathbb{E}_{s\sim\nu^\pi,a\sim\pi_\theta}
\begin{bmatrix}
R
\end{bmatrix}$$
> 策略是 **随机性** 的，计算需要依赖于 **概率性动作分布：$a\sim\pi_{\theta}(\cdot|s)$**，这在面对 **连续动作空间** 时变得非常复杂
> **确定性策略梯度定理（Deterministic Policy Gradient Theorem, DPG 定理）** 假设有一个 **确定性策略 $\mu$**，**直接输出动作：$a=\mu_\theta(s)$**，而不是动作的概率分布：$a\sim\pi_{\theta}(\cdot|s)$
> 此时：
$$J(\mu_\theta)=\int_{\mathcal{S}}\nu^{\mu_\theta}(s)r(s,\mu_\theta(s))\mathrm{d}s=\mathbb{E}_{s\sim\nu^{\mu_\theta}}[r(s,\mu_\theta(s))]$$
> 可以避免计算**概率分布**的导数，简化梯度计算过程
> **离线策略形式**：
$$J(\theta)=\int_{\mathcal{S}}\nu^{\pi_\beta} V^{\mu_\theta}(s)\mathrm{d}s=\int_{\mathcal{S}}\nu^{\pi_\beta} Q^{\mu_\theta}(s,\mu_\theta(s))\mathrm{d}s=\mathbb{E}_{s\sim\nu^{\pi_\beta}}
\begin{bmatrix}
Q^{\mu_\theta}(s,a)|_{a=\mu_\theta(s)}
\end{bmatrix}$$
- $\pi_{\beta}$是用来收集数据的行为策略
> 其 **梯度** 为（证明过程与 **策略梯度定理** 相似，详细证明过程可看原始论文[Deterministic Policy Gradient Algorithms](https://proceedings.mlr.press/v32/silver14.pdf)）：
$$\nabla_\theta J(\pi_\theta)=\mathbb{E}_{s\sim\nu^{\pi_\beta}}
\begin{bmatrix}
\nabla_\theta\mu_\theta(s)\nabla_aQ_\omega^{\mu_\theta}(s,a)|_{a=\mu_\theta(s)}
\end{bmatrix}$$
- 可见，$Q_\omega$ 就是**Critic**，$\mu_\theta$ 就是**Actor**，这是一个 **Actor-Critic** 的框架

### DDPG 中的目标网络
> **DDPG** 要用到4个神经网络，其中 **Actor** 和 **Critic** 各用一个网络，此外还有各自对应的： **目标网络**$\mu_{\theta^{\prime}}$ 和 $Q_{\omega^{\prime}}$（为了 **稳定性**）
> **DDPG** 中目标网络的更新与 **DQN** 中略有不同：在 **DQN** 中，**每隔一段时间**将网络**直接复制**给目标网络；而在 **DDPG** 中，目标网络的更新采取的是一种 **软更新（soft update）** 的方式，即让 **目标网络** 缓慢更新，逐渐接近 **主网络**：

$$\theta^{\prime}\leftarrow\tau\theta+(1-\tau)\theta^{\prime}$$
$$[\omega^{\prime}\leftarrow\tau\omega+(1-\tau)\omega^{\prime}]$$

- $\theta^{\prime}$ 是 **目标网络** 的参数
- $\theta$ 是 **主网络** 的参数
- $\tau$是 **软更新** 的步长，一般设置为一个小的值，如 0.001


### 其他细节
1. 由于 Q函数 存在 **值过高估计** 的问题，**DDPG** 采用了 **Double DQN** 中的技术来更新网络
2. 由于 **DDPG** 使用的是 **确定性策略**，即每个状态下都会输出一个确定的动作，这可能会导致模型在训练过程中过度依赖当前策略而陷入 **局部最优解**，缺乏多样性和探索。因此，引入 **随机噪声（Random Noise）$\mathcal{N}$** 来鼓励探索（常用的噪声类型包括 **Ornstein-Uhlenbeck 噪声** 和 **高斯噪声**）：

$$a_t=\mu(s_t)+\mathcal{N}_t$$
> **DDPG** 最常用的噪声（原始论文）是 **Ornstein-Uhlenbeck 噪声（OU 噪声）** ，适用于连续控制任务，能够生成 **平滑的、具有时序相关性** 的噪声，适合连续动作空间中的探索：
$$x_{t+1}=\theta(\mu-x_t)+\sigma\cdot\mathcal{N}(0,1)$$

- $\mu$是噪声的均值 (通常为0)
- $\theta$是回归速率，控制噪声回归到均值的速度
- $\sigma$是噪声的强度，控制噪声的幅度
- $\mathcal{N}(0,1)$是标准正态分布噪声
- 表现为向均值靠拢，适用于有惯性的系统

> **高斯噪声** 适用于简单的任务，生成方式非常简单，但在连续空间中通常不如 **OU 噪声** 平滑：
$$a_t=\mu(s_t)+\mathcal{N}(0,\sigma)$$

- $a_t$是动作，$\mu(s_t)$是从 Actor 网络输出的动作，$\mathcal{N}(0,\sigma)$是从标准正态分布中生成的噪声

## 16.2 DDPG 代码实践（Pendulum-v1）

#### DDPG 算法伪代码：

##### 初始化
- 初始化 Actor 网络和 Critic 网络
- 初始化目标网络 (Actor_target, Critic_target)
- 初始化经验回放缓冲区

##### 每个训练回合
1. 初始化状态 `s`

2. **每个时间步 `t`**：
   - 使用 Actor 网络选择动作 `a`，并加上探索噪声：`a = actor(s) + 噪声`
   - 执行动作 `a`，观察下一个状态 `s'` 和奖励 `r`
   - 将经验 `(s, a, r, s')` 存储到经验回放缓冲区

3. **从经验回放中采样一批数据**：
   - 从回放缓冲区中随机采样：`(s_batch, a_batch, r_batch, s'_batch)`

4. **更新 Critic 网络**：
   - 计算目标 Q 值：`y = r_batch + gamma * critic_target(s'_batch, actor_target(s'_batch))`
   - 最小化误差：`Q(s, a) - y`，更新 Critic 网络

5. **更新 Actor 网络**：
   - 计算策略梯度：`∇_a Q(s, a)` 的梯度
   - 使用策略梯度更新 Actor 网络参数

6. **软更新目标网络**：
   - 使用软更新公式：`目标网络参数 = tau * 当前网络参数 + (1 - tau) * 目标网络参数`

##### end


导入相关库：

In [7]:
# 基本库
import numpy as np

from utils.replay_buffer import ReplayBuffer
from utils.training import train_off_policy_agent
from utils.smoothing import moving_average
# 神经网络
import torch
import torch.nn.functional as F
# Gymnasium 是一个用于开发和测试强化学习算法的工具库，为 OpenAI Gym 的更新版本（2021迁移开发）
import gymnasium as gym

ImportError: cannot import name 'train_off_policy_agent' from 'utils.training' (D:\GitHub\SH-RL-notes\RL_Advanced_Algorithms\utils\training.py)

### 定义策略网络和价值网络:

- 一层隐藏层
- **策略网络** 的输出层用正切函数$(y=\tanh x)$作为激活函数，值域是[-1,1]，方便按比例调整成环境可以接受的动作范围
- **$Q$网络** 的输入是状态和动作拼接后的向量，**$Q$网络** 的输出是一个值，表示该状态动作对的价值。

In [ ]:
class PolicyNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim, action_bound):
        super(PolicyNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, action_dim)
        self.action_bound = action_bound  # action_bound是环境可以接受的动作最大值

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return torch.tanh(self.fc2(x)) * self.action_bound


class QValueNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(QValueNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim + action_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, hidden_dim)
        self.fc_out = torch.nn.Linear(hidden_dim, 1)

    def forward(self, x, a):
        cat = torch.cat([x, a], dim=1) # 拼接状态和动作
        x = F.relu(self.fc1(cat))
        x = F.relu(self.fc2(x))
        return self.fc_out(x)

### 定义 DDPG 算法：
> 使用 **高斯噪音**

In [ ]:
class DDPG:
    def __init__(self, state_dim, hidden_dim, action_dim, action_bound, sigma, actor_lr, critic_lr, tau, gamma, device):
        
        self.actor = PolicyNet(state_dim, hidden_dim, action_dim, action_bound).to(device)
        self.critic = QValueNet(state_dim, hidden_dim, action_dim).to(device)
        self.target_actor = PolicyNet(state_dim, hidden_dim, action_dim, action_bound).to(device)
        self.target_critic = QValueNet(state_dim, hidden_dim, action_dim).to(device)
        
        # 初始化目标网络并设置和主网络相同的参数
        self.target_critic.load_state_dict(self.critic.state_dict())
        self.target_actor.load_state_dict(self.actor.state_dict())
        
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=actor_lr)
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=critic_lr)
        
        self.gamma = gamma
        self.sigma = sigma  # 高斯噪声的标准差设为0
        self.tau = tau  # 目标网络软更新参数
        self.action_dim = action_dim
        self.device = device

    def take_action(self, state):
        state = torch.tensor(np.array([state]), dtype=torch.float).to(self.device)
        action = self.actor(state).item()
        # 给动作添加噪声，增加探索
        action = action + self.sigma * np.random.randn(self.action_dim)
        return action

    # 软更新
    def soft_update(self, net, target_net):
        for param_target, param in zip(target_net.parameters(), net.parameters()):
            param_target.data.copy_(param_target.data * (1.0 - self.tau) + param.data * self.tau)

    def update(self, transition_dict):
        
        states_np = np.array(transition_dict['states'])  # 转换成统一的大 np.ndarray，PyTorch更高效处理
        states = torch.tensor(states_np, dtype=torch.float).to(self.device)
        actions = torch.tensor(transition_dict['actions']).view(-1, 1).to(self.device)
        rewards = torch.tensor(transition_dict['rewards'], dtype=torch.float).view(-1, 1).to(self.device)
        next_states_np = np.array(transition_dict['next_states'])  # 转换成统一的大 np.ndarray，PyTorch更高效处理
        next_states = torch.tensor(next_states_np, dtype=torch.float).to(self.device)
        dones = torch.tensor(transition_dict['dones'], dtype=torch.float).view(-1, 1).to(self.device)


        next_q_values = self.target_critic(next_states, self.target_actor(next_states))
        q_targets = rewards + self.gamma * next_q_values * (1 - dones)
        
        critic_loss = torch.mean(F.mse_loss(self.critic(states, actions), q_targets))
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        actor_loss = -torch.mean(self.critic(states, self.actor(states)))
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        self.soft_update(self.actor, self.target_actor)  # 软更新策略网络
        self.soft_update(self.critic, self.target_critic)  # 软更新价值网络

### 环境设置（'Pendulum-v1'）：

In [ ]:
np.random.seed(0)    # 设置 NumPy 的随机种子
torch.manual_seed(0) # 设置 PyTorch CPU 随机种子
torch.cuda.manual_seed_all(0) # 设置 PyTorch GPU 随机种子, 由于GPU并行性, 只能极大减小偏差

env_name = 'Pendulum-v1'
env = gym.make(env_name)
env.reset(seed=0)   # 环境通常依赖于其他随机数生成器来初始化状态、进行探索(推荐位于以上随机之后)
print("Environment spec:", env.spec)

### 超参数设置：

In [ ]:
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]  # 连续动作空间
action_bound = env.action_space.high[0]  # 动作最大值
hidden_dim = 64

actor_lr = 3e-4
critic_lr = 3e-3
tau = 0.005  # 软更新参数
gamma = 0.98
sigma = 0.01  # 高斯噪声标准差
device = torch.device("cuda") if torch.cuda.is_available() else torch.device( "cpu")

buffer_size = 10000
minimal_size = 1000
batch_size = 64
replay_buffer = ReplayBuffer(buffer_size)

agent = DDPG(state_dim, hidden_dim, action_dim, action_bound, sigma, actor_lr, critic_lr, tau, gamma, device)

num_episodes = 200

In [ ]:
return_list = train_off_policy_agent(env, agent, num_episodes, replay_buffer, minimal_size, batch_size)